# Testing Multiple Models since our dataset is ready

In [1]:
import os 
import pandas as pd
import numpy as np
import tensorflow as tf 
import shutil
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from glob import glob
from PIL import Image

import ssl

ssl._create_default_https_context = ssl._create_unverified_context


In [2]:
#Setting Global Variables
FINAL_PATH = "dataset_final//imagens//"

TRAIN_PATH = "dataset_final//train//"

TEST_PATH = "dataset_final//test//"

SIZE=224

In [3]:
pele_df = pd.read_csv('dataset_final/classificacao_dataset.csv')


le = LabelEncoder()
le.fit(pele_df['type'])
LabelEncoder()
print('label 0 é igual á ' + list(le.classes_)[0])
print('label 1 é igual á ' + list(le.classes_)[1])

pele_df['label'] = le.transform(pele_df["type"]) 


print(pele_df['label'].value_counts())

print(pele_df.head())

label 0 é igual á mel
label 1 é igual á nv
1    6805
0    1183
Name: label, dtype: int64
   Unnamed: 0      image_id type  label
0           0  ISIC_0024698   nv      1
1           1  ISIC_0024693   nv      1
2           2  ISIC_0025964  mel      0
3           3  ISIC_0030623  mel      0
4           4  ISIC_0027190  mel      0


In [5]:
pele_df_final = pele_df.copy()

image_path = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join('dataset_final', '*', '*.jpg'))}

#Definindo o diretório como uma nova coluna
pele_df_final['path'] = pele_df['image_id'].map(image_path.get)

#Usar o dataframe completo estava levando muito tempo, portanto tive de quebra-lo num fração menor, ainda com mais de 2000 linhas
pele_df_final_frac = pele_df_final.sample(frac=0.35)

print(pele_df_final_frac['type'].value_counts())

X = pele_df_final_frac['path']
Y = pele_df_final_frac['type'] #Assign label values to Y

nv     2395
mel     401
Name: type, dtype: int64


In [8]:
print(f'Valores únicos para Y: {list(set(Y))}')

x_train_auto, x_test_auto, y_train_auto, y_test_auto = train_test_split(X, Y, test_size=0.20, random_state=42)

print(f'x_train_auto: {x_train_auto[0:5]}')
print(f'x_test_auto: {x_test_auto[0:5]}')
print(f'y_train_auto: {y_train_auto[0:5]}')
print(f'y_test_auto: {y_test_auto[0:5]}')

Valores únicos para Y: ['nv', 'mel']
x_train_auto: 972     dataset_final/imagens/ISIC_0028567.jpg
7863          dataset_final/imagens/586355.jpg
7838          dataset_final/imagens/363475.jpg
4873    dataset_final/imagens/ISIC_0027264.jpg
1461    dataset_final/imagens/ISIC_0028539.jpg
Name: path, dtype: object
x_test_auto: 6180    dataset_final/imagens/ISIC_0033390.jpg
4795    dataset_final/imagens/ISIC_0028417.jpg
7595    dataset_final/imagens/ISIC_0033247.jpg
207     dataset_final/imagens/ISIC_0024496.jpg
3513    dataset_final/imagens/ISIC_0026063.jpg
Name: path, dtype: object
y_train_auto: 972     mel
7863    mel
7838    mel
4873     nv
1461     nv
Name: type, dtype: object
y_test_auto: 6180     nv
4795     nv
7595     nv
207     mel
3513     nv
Name: type, dtype: object


In [6]:
df_train = pd.DataFrame(columns=['image_path','type'])
df_train['image_path'] = x_train_auto
df_train['type'] = y_train_auto

df_test = pd.DataFrame(columns=['image_path','type'])
df_test['image_path'] = x_test_auto
df_test['type'] = y_test_auto

In [7]:
#Creating a directory for Train 
for n, row in df_train.iterrows():
    if row['type']=='mel':
        shutil.copy2(row['image_path'], TRAIN_PATH + 'mel//')
    else: 
        shutil.copy2(row['image_path'], TRAIN_PATH+ 'nv//')

In [8]:
#Creating a directory for Test
for n, row in df_test.iterrows():
    if row['type']=='mel':
        shutil.copy2(row['image_path'], TEST_PATH + 'mel//')
    else: 
        shutil.copy2(row['image_path'], TEST_PATH+ 'nv//')

## Testing VGG-16

In [9]:
SIZE = 224

In [10]:
# Image Augmentation

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [11]:
# Training and Validation Sets
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(TRAIN_PATH, batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(TEST_PATH,  batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

Found 2236 images belonging to 2 classes.
Found 560 images belonging to 2 classes.


In [12]:
# Loading the Base Model
from keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (SIZE, SIZE, 3), # Shape of our images
include_top = True, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in base_model.layers:
    print(layer)
    layer.trainable = False

2022-03-28 19:52:34.491616: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Compile and Fit

# Flatten the output layer to 1 dimension
print(layers.Flatten()(base_model.output))
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

ValueError: The name "flatten" is used 2 times in the model. All layer names should be unique.

In [ ]:
vgghist = model.fit(train_generator, validation_data = validation_generator, batch_size=20, steps_per_epoch = 20, epochs = 8)

## Testing Inception

In [14]:
SIZE = 150

In [6]:
# Data Augmentation
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [8]:
# Training and Validation Generators

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(TRAIN_PATH, batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(TEST_PATH,  batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

Found 2236 images belonging to 2 classes.
Found 560 images belonging to 2 classes.


In [17]:
# Loading the Base Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape = (SIZE, SIZE, 3), include_top = False, weights = 'imagenet')

In [18]:
# Compile and Fit

for layer in base_model.layers:
    layer.trainable = False

from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)
x = layers.Dense(18432, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [19]:
inc_history = model.fit(train_generator, validation_data = validation_generator, batch_size=20, steps_per_epoch = 20, epochs = 8)

Epoch 1/8
20/20 [==============================] - 424s 19s/step - loss: 5.5184 - acc: 0.7250 - val_loss: 1.0225 - val_acc: 0.8339
Epoch 2/8
20/20 [==============================] - 254s 13s/step - loss: 1.6929 - acc: 0.8075 - val_loss: 2.7590 - val_acc: 0.8304
Epoch 3/8
20/20 [==============================] - 220s 11s/step - loss: 1.1718 - acc: 0.8525 - val_loss: 1.2351 - val_acc: 0.6732
Epoch 4/8
20/20 [==============================] - 837s 44s/step - loss: 1.6155 - acc: 0.8200 - val_loss: 0.7700 - val_acc: 0.8321
Epoch 5/8
20/20 [==============================] - 190s 9s/step - loss: 1.0785 - acc: 0.8225 - val_loss: 0.5307 - val_acc: 0.8321
Epoch 6/8
20/20 [==============================] - 208s 10s/step - loss: 1.1973 - acc: 0.8025 - val_loss: 0.5526 - val_acc: 0.8196
Epoch 7/8
20/20 [==============================] - 342s 18s/step - loss: 1.5651 - acc: 0.7675 - val_loss: 0.6969 - val_acc: 0.7679
Epoch 8/8
20/20 [==============================] - 210s 10s/step - loss: 0.8909 - ac

## Testing ResNet50

In [ ]:
SIZE=224

In [ ]:
# Data Augmentation and Generators
# Add our data-augmentation parameters to ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(TRAIN_PATH, batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

validation_generator = test_datagen.flow_from_directory(TEST_PATH, batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

In [ ]:
#Import the base model
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(SIZE, SIZE,3), include_top=False, weights="imagenet")

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Build and Compile the Model
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(1, activation='sigmoid'))

#opt = keras.optimizer_v2.gradient_descent.SGD(learning_rate=0.0001)
base_model.compile(optimizer = 'SGD', loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
resnet_history = base_model.fit(train_generator, batch_size=20, validation_data = validation_generator, steps_per_epoch = 20, epochs = 8)

## Testing EfficientNet

In [ ]:
!pip install -U efficientnet_3D

In [ ]:
import efficientnet_3D.tfkeras as efn
keras.utils.generic_utils = keras.utils.generic_utils

In [ ]:
SIZE = 224

In [ ]:
# Image Augmentation
# Add our data-augmentation parameters to ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(TRAIN_PATH, batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

validation_generator = test_datagen.flow_from_directory(TEST_PATH, batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))


In [ ]:
# Loading the Base Model
base_model = efn.EfficientNetB0(input_shape = (SIZE, SIZE, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Build the Model
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(input = base_model.input, output = predictions)

In [ ]:
# Compile and Fit
model_final.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])

eff_history = model_final.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 20, epochs = 8)

# Testing Random Forest Classifier

In [ ]:
SIZE = 224

Error: Session cannot generate requests

In [ ]:
# Data Augmentation and Generators
# Add our data-augmentation parameters to ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(TRAIN_PATH, batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

validation_generator = test_datagen.flow_from_directory(TEST_PATH, batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

Error: Session cannot generate requests

In [13]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

nsamples, nx, ny, nrgb = train_generator.shape
x_train2 = train_generator.reshape((nsamples,nx*ny*nrgb))

nsamples, nx, ny, nrgb = validation_generator.shape
x_test2 = validation_generator.reshape((nsamples,nx*ny*nrgb))

AttributeError: 'DirectoryIterator' object has no attribute 'shape'

In [10]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()

In [16]:
model.fit(train_generator,validation_generator)

In [ ]:
y_pred=model.predict(x_test2)
y_pred

Error: Session cannot generate requests

In [ ]:
accuracy_score(y_pred,y_test)
print(classification_report(y_pred,y_test))